
## Time series analysis (trend, slope, ... calculation) for JEDI-CUBES
This notebook was set up to help with the analysis of time series in jedi cubes. It can be connected directly to JEDI cubes and examine a time series contained there for various reference units (NUTS3,2,1,0, 1km, 10km, ....). If you have any questions, please contact: loehnertz@space4environment.com




(1) connection to jedi cube tables on MS-SQL (TEAL and GREENMONKEY)
!Please note: you need read and partially write permissions for the database to be used.!

Hallo Eva und Maria ..wieder was neues

In [4]:
# Reading libaries and start a test-connection to different MS-SQL Server:
import pyodbc 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy as sa
from sqlalchemy import create_engine, event, text
from sqlalchemy.engine.url import URL
import json

from scipy import stats

#check processing time:##
import time  
import datetime
start_time = time.time()
##########################

#### GREENMONKEY

print ("connect to engine GREENMONKEY")
engine_GREENMONKEY = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
connection = engine_GREENMONKEY.raw_connection()
cursor = connection.cursor()
print ("connection to greenmonkey found")
print ("the function HAS_DBACCESS returns 1 if the user has access to the database, otherwise, it will return as 0")  
#

with engine_GREENMONKEY.begin() as conn:
      query_check=text('''       
      SELECT [admin_category]
      FROM [drought].[SMA_GDMP_cube_2]
            ''')
      readable_database = pd.read_sql_query(query_check, conn)
      # cursor.execute(query_check)
      # cursor.close()
##connection.commit()
#print ("following databases can be READ:")
##print (readable_database)
#print ("----------------------------------------------------------------")
#
#
#
#

print ("end....")
print("--- %s seconds ---" % (time.time() - start_time))
seconds = time.time() - start_time
print('Time Taken: HH:MM:SS ', time.strftime("%H:%M:%S",time.gmtime(seconds)))

connect to engine GREENMONKEY
connection to greenmonkey found
the function HAS_DBACCESS returns 1 if the user has access to the database, otherwise, it will return as 0
       
SELECT [admin_category]
FROM [drought].[SMA_GDMP_cube_2]
      
end....
--- 1.94398832321167 seconds ---
Time Taken: HH:MM:SS  00:00:01


### (1) reading CUBE 2 

[Climate_Impact].[drought].[SMA_GDMP_cube_2]

In [6]:
## connect to MS-sql  



## testing connection with simple query:
print ("Reading drought table from greenmonkey")
query_drought_cube2=text('''
SELECT  top (100)
    [admin_category]
      ,[GridNum10km]
      ,[LULUCF_CODE]
      ,[env_zones]
      ,[natura2000_protection]
      ,[Areaha]
     
      ,[annual_drought_productivity_deficit_2000]
      ,[annual_drought_productivity_deficit_2001]
      ,[annual_drought_productivity_deficit_2002]
      ,[annual_drought_productivity_deficit_2003]
      ,[annual_drought_productivity_deficit_2004]
      ,[annual_drought_productivity_deficit_2005]
      ,[annual_drought_productivity_deficit_2006]
      ,[annual_drought_productivity_deficit_2007]
      ,[annual_drought_productivity_deficit_2008]
      ,[annual_drought_productivity_deficit_2009]
      ,[annual_drought_productivity_deficit_2010]
      ,[annual_drought_productivity_deficit_2011]
      ,[annual_drought_productivity_deficit_2012]
      ,[annual_drought_productivity_deficit_2013]
      ,[annual_drought_productivity_deficit_2014]
      ,[annual_drought_productivity_deficit_2015]
      ,[annual_drought_productivity_deficit_2016]
      ,[annual_drought_productivity_deficit_2017]
      ,[annual_drought_productivity_deficit_2018]
      ,[annual_drought_productivity_deficit_2019]
      ,[annual_drought_productivity_deficit_2020]
      ,[annual_drought_productivity_deficit_2021]
      ,[annual_drought_productivity_deficit_2022]
  FROM [Climate_Impact].[drought].[SMA_GDMP_cube_2]


''')

with engine_GREENMONKEY.begin() as conn:
  df_drought_cube2 = pd.read_sql_query(query_drought_cube2, conn) ###### import table into a dataframe (df) for the use in pandas!!!!!!!!!!!



### update GridNum10km .. set float to INT
df_drought_cube2['GridNum10km'] = df_drought_cube2['GridNum10km'].astype('Int64')
df_drought_cube2



print (" sql table as data frame for the next steps:")
print (df_drought_cube2)



## close connection:
#cursor.close(
#connection.commit()




print ("reading cube 2 done")

Reading drought table from greenmonkey
 sql table as data frame for the next steps:
    admin_category        GridNum10km LULUCF_CODE env_zones  \
0              750   9439333094260736          CL       MDS   
1             1439   9439874260140032          FL       MDS   
2              365   9454094896857088          FL       MDM   
3              366   9454842221166592          CL       MDS   
4              366   9455078444367872          FL       MDS   
..             ...                ...         ...       ...   
95             457  14219124888567808          CL       ALS   
96             458  14219992471961600          FL       ATC   
97             458  14220056896471040          CL       ALS   
98            1458  14220147090784256          FL       ATC   
99             421  14223909482135552          CL       ATC   

          natura2000_protection  Areaha  \
0   none Nature 2000 protection  1931.0   
1                    Natura2000   854.0   
2                    Natura200

In [9]:
df_annual_drought_productivity_deficit = df_drought_cube2

print ("table read from TEAL job done")
## Tronsforme the table columns to rows:
df_transformed =df_annual_drought_productivity_deficit.melt(id_vars=['admin_category', 'GridNum10km', 'LULUCF_CODE', 'env_zones', 'natura2000_protection','Areaha'], 
     var_name="year_string", 
       value_name="annual_drought_productivity_deficit")

print ("table TRANSFORMED -pivot- job done")


#Convert the Strings to Datetime in the DataFrame
count_row = df_transformed.shape[0]  # Gives number of rows
count_col = df_transformed.shape[1]  # Gives number of columns#

print ('number of rows: ' + str(count_row))
print ('number of columns: ' + str(count_col))


#cleaning the table: change the year from string to INT or DATE-format:
df_transformed['year_string_number'] = df_transformed['year_string'].str[-4:]
df_transformed['year'] = df_transformed['year_string_number'].astype(int)
df_transformed.drop(['year_string_number'], axis=1)
#df_transformed['year'] = pd.to_datetime(df_transformed['year'], format='%Y')
print ("year value updated done")


######################################## calcualtion slope,...
# 
# ## group ############################################################### =['Admin', 'Env', 'CLC00', 'CLC18','AreaHa']
# 
# ## here we are doing a groupby -(in the moment we have also lines for CLC06, CL12,) --> group by can also be done during SQL reading
dfg = df_transformed.groupby(["admin_category","GridNum10km","LULUCF_CODE","env_zones","natura2000_protection","year"]).agg({'Areaha' : ['sum'], 'annual_drought_productivity_deficit' : ['mean']}).reset_index()  # this function do a group by and calc. the sum area and min,max,avg for LINT
# #cleaning columns:
dfg.columns = ["_".join(x) for x in dfg.columns.ravel()]
dfg = dfg.rename(columns={'admin_category_': 'admin_category', 'GridNum10km_': 'GridNum10km', 'LULUCF_CODE_': 'LULUCF_CODE', 
                          'env_zones_': 'env_zones', 'natura2000_protection_': 'natura2000_protection','year_': 'year'}).reset_index()#.sort_values(ascending=True)
# # calculate weighted AVG:
# #dfg['LINT_anom_values_weightavg'] =dfg['AreaHa_sum'] * dfg['LINT_anom_values_mean']   ## not needed for C-tables 
# 


table read from TEAL job done
table TRANSFORMED -pivot- job done
number of rows: 2300
number of columns: 8
year value updated done


Q:\UserTemp\ricci\AppData\Local\Temp\8\ipykernel_23572\3141846633.py:35: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  dfg.columns = ["_".join(x) for x in dfg.columns.ravel()]


In [17]:
dfg.groupby(["admin_category", "GridNum10km","LULUCF_CODE","env_zones","natura2000_protection"]).agg({
    'year': [lambda x: list(x)],
    'annual_drought_productivity_deficit_mean': [lambda x: list(x)]})

year  \
                                                                                                                             <lambda>   
admin_category GridNum10km       LULUCF_CODE env_zones natura2000_protection                                                            
29             14078494774394880 GL          LUS       Natura2000                   [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
327            9732949943517184  GL          LUS       none Nature 2000 protection  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
               9732962828419072  SL          LUS       none Nature 2000 protection  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
329            9733435274821632  FL          LUS       none Nature 2000 protection  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
               14077219169107968 GL          LUS       none Nature 2000 protection  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
...                                                                                                                               ...   
1450           14171162988773376 FL          ATC       none Nature 2000 protection  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
1451           14207219239223296 CL          ATC       none Nature 2000 protection  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
1457           14200300046909440 FL          MDM       none Nature 2000 protection  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
               14201498342785024 CL          MDM       none Nature 2000 protection  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
1458           14220147090784256 FL          ATC       none Nature 2000 protection  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   

                                                                                             annual_drought_productivity_deficit_mean  
                                                                                                                             <lambda>  
admin_category GridNum10km       LULUCF_CODE env_zones natura2000_protection                                                           
29             14078494774394880 GL          LUS       Natura2000                   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
327            9732949943517184  GL          LUS       none Nature 2000 protection  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -62695.41601562...  
               9732962828419072  SL          LUS       none Nature 2000 protection  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
329            9733435274821632  FL          LUS       none Nature 2000 protection  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
               14077219169107968 GL          LUS       none Nature 2000 protection  [0.0, 0.0, 0.0, -101647.837890625, 0.0, 0.0, 0...  
...                                                                                                                               ...  
1450           14171162988773376 FL          ATC       none Nature 2000 protection  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1451           14207219239223296 CL          ATC       none Nature 2000 protection  [0.0, 0.0, 0.0, 0.0, 0.0, -1223.32421875, 0.0,...  
1457           14200300046909440 FL          MDM       none Nature 2000 protection  [0.0, 0.0, 0.0, -332161.01953125, 0.0, 0.0, 0....  
               14201498342785024 CL          MDM       none Nature 2000 protection  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1458           14220147090784256 FL          ATC       none Nature 2000 protection  [0.0, 0.0, 0.0, -154634.46875, 0.0, 0.0, 0.0, ...  

[100 rows x 2 columns]

In [28]:

# ################################################################################# model start
# # model for calculation trend:  https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
def model_trend_stat(row):
#    # get variables for linear fitting
    y = list(row.annual_drought_productivity_deficit_mean)
    x = list(row.year) # make sure years are in ascending order
  
#    # linear regression
    m = stats.linregress(x,y)
    slope =m.slope 
    intercept=m.intercept
    rvalue=m.rvalue
    pvalue=m.pvalue
    stderr=m.stderr
    x.sort()
    y_fitted = [m.slope * i + m.intercept for i in x]
    y_fit_start = y_fitted[0]
    y_fit_end =   y_fitted[-1]

   #relative_change = (y_end - y_start) / y_start  ### cehck divide by zero
    if y_fit_start != 0: 
        relative_change = (y_fit_end - y_fit_start) / np.abs(y_fit_start) *100
    else:
        relative_change = 0
    # return x.sort()
    return pd.Series({'rvalue': rvalue, 'slope':slope,'intercept':intercept,'pvalue':pvalue,'stderr':stderr, 
                    'yearMIN': x[0],'yearMAX': x[-1],'y_start': y_fit_start,'y_end': y_fit_end,'relative_change': relative_change})
################################################################################ model end
# admin_category","GridNum10km","LULUCF_CODE","env_zones","natura2000_protection"
# #calcualtion of slope, trend,... for every group : (reference unit):
dfg_trend= dfg.groupby(["admin_category", "GridNum10km","LULUCF_CODE","env_zones","natura2000_protection"]).apply(model_trend_stat).reset_index()#.sort_values(ascending=False)

##dfg_trend= dfg.groupby(["Admin", "Env","CLC00","CLC18"]).apply(model_trend_stat).reset_index()#.sort_values(ascending=False)

#print (df_transformed)
print("--- %s seconds ---" % (time.time() - start_time))
seconds = time.time() - start_time
print('Time Taken: HH:MM:SS ', time.strftime("%H:%M:%S",time.gmtime(seconds)))
print ("end....")

--- 1317.9049727916718 seconds ---
Time Taken: HH:MM:SS  00:21:57
end....


In [29]:
dfg_trend

,admin_category,GridNum10km,LULUCF_CODE,env_zones,natura2000_protection,rvalue,slope,intercept,pvalue,stderr,yearMIN,yearMAX,y_start,y_end,relative_change
0,29,14078494774394880,GL,LUS,Natura2000,-0.192847,-0.653196,1.308787e+03,0.377991,0.725255,2000.0,2022.0,2.395051,-11.975257,-600.000000
1,327,9732949943517184,GL,LUS,none Nature 2000 protection,0.157263,302.856212,-6.118457e+05,0.473606,415.014858,2000.0,2022.0,-6133.247219,529.589440,108.634732
2,327,9732962828419072,SL,LUS,none Nature 2000 protection,-0.007720,-2.994957,5.106376e+03,0.972113,84.657132,2000.0,2022.0,-883.538150,-949.427204,-7.457409
3,329,9733435274821632,FL,LUS,none Nature 2000 protection,0.000000,0.000000,0.000000e+00,1.000000,0.000000,2000.0,2022.0,0.000000,0.000000,0.000000
4,329,14077219169107968,GL,LUS,none Nature 2000 protection,0.257130,803.540220,-1.620339e+06,0.236244,659.010297,2000.0,2022.0,-13258.413638,4419.471213,133.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1450,14171162988773376,FL,ATC,none Nature 2000 protection,-0.466461,-124.109419,2.489841e+05,0.024853,51.356870,2000.0,2022.0,765.306110,-1965.101117,-356.773217
96,1451,14207219239223296,CL,ATC,none Nature 2000 protection,-0.396233,-372.711818,7.477464e+05,0.061242,188.463408,2000.0,2022.0,2322.733271,-5876.926715,-353.017718
97,1457,14200300046909440,FL,MDM,none Nature 2000 protection,0.225264,2299.882697,-4.640819e+06,0.301397,2170.676392,2000.0,2022.0,-41053.123896,9544.295431,123.248646
98,1457,14201498342785024,CL,MDM,none Nature 2000 protection,-0.353553,-402.893576,8.086074e+05,0.097929,232.610715,2000.0,2022.0,2820.255031,-6043.403639,-314.285714


Export data frame to greenmonkey:


In [17]:
# #exporting data to GREENMONKEY#################################################################################
print ("connect to engine GREENMONKEY")  #drought
engine_GREENMONKEY = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
name_of_table ="Drought_trend_xxxxxxxx"       
print ("connect to greenmonkey engine.....GREENMONKEY and store sql table")  
dfg_trend.to_sql(name_of_table, engine_GREENMONKEY, if_exists='replace', index = False, schema='drought')
# 
# ##################################################################################################################

connect to engine GREENMONKEY
connect to greenmonkey engine.....GREENMONKEY and store sql table


91